In [ ]:
# ✅ INSTALLATION
!pip install PyMuPDF fpdf langchain-community langchain-groq faiss-cpu sentence-transformers scikit-learn gradio httpx

# ✅ IMPORTS
import os
import gradio as gr
import fitz  # PyMuPDF
from fpdf import FPDF
import httpx
import asyncio
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor
import tempfile

# ✅ CONFIG
GROQ_API_KEY = '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'  # Replace this
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
MODEL_NAME = "llama3-8b-8192"
MAX_MEMORY_ITEMS = 4

conversation_memory = []
text_embedder = SentenceTransformer("all-MiniLM-L6-v2")

# ✅ KEYWORDS FOR EDUCATIONAL FILTERING
EDU_KEYWORDS = {
    # Core educational indicators - must have these
    "chapter", "syllabus", "curriculum", "lesson", "module", "unit",
    "course", "textbook", "workbook", "assignment", "homework",
    "quiz", "test", "exam", "examination", "exercise", "question",
    "learning objective", "study guide", "practice problems",
    "lecture notes", "tutorial", "educational", "academic"
}

# Strong educational phrases that are more specific
EDU_PHRASES = {
    "chapter 1", "chapter 2", "chapter 3", "chapter 4", "chapter 5",
    "learning objectives", "study questions", "practice exercises",
    "review questions", "end of chapter", "homework problems",
    "assignment instructions", "course syllabus", "lesson plan",
    "educational goals", "learning outcomes", "study materials",
    "academic content", "textbook exercises", "quiz questions"
}

def is_educational(text):
    if not text or len(text.strip()) < 100:  # Too short to be educational
        return False

    lowered = text.lower()

    # Check for educational phrases first (more specific)
    phrase_count = sum(1 for phrase in EDU_PHRASES if phrase in lowered)

    # Check for educational keywords
    keyword_count = sum(1 for keyword in EDU_KEYWORDS if keyword in lowered)

    # Print debug info to see what's being detected
    print(f"DEBUG: Educational phrases found: {phrase_count}")
    print(f"DEBUG: Educational keywords found: {keyword_count}")
    print(f"DEBUG: Text length: {len(text)}")

    # Must have at least 1 educational phrase OR at least 5 educational keywords
    # This makes it much more strict
    is_edu = phrase_count >= 1 or keyword_count >= 5

    print(f"DEBUG: Is educational: {is_edu}")
    return is_edu

# ✅ MEMORY MGMT
def clear_conversation_memory():
    global conversation_memory
    conversation_memory = []

# ✅ PDF TEXT EXTRACTION
def extract_text_from_pdf(file_bytes):
    try:
        with fitz.open(stream=BytesIO(file_bytes), filetype="pdf") as doc:
            return [page.get_text() for page in doc]
    except Exception as e:
        print(f"Error: {e}")
        return []

def chunk_text(text, chunk_size=300):
    if not text.strip(): return []
    sentences = text.split(". ")
    chunks, current = [], ""
    for s in sentences:
        current += s.strip() + ". "
        if len(current) > chunk_size:
            chunks.append(current.strip())
            current = ""
    if current.strip(): chunks.append(current.strip())
    return chunks

# ✅ CLASSIFY & CHUNK PDF

def process_uploaded_pdfs(uploaded_files):
    try:
        if not uploaded_files:
            return "❌ No files uploaded", None

        file_bytes_list = [file if isinstance(file, bytes) else file.read() for file in uploaded_files]

        with ThreadPoolExecutor() as executor:
            texts = list(executor.map(extract_text_from_pdf, file_bytes_list))

        valid_texts = [text for text in texts if text]
        all_text = "\n\n".join(["\n".join(pages) for pages in valid_texts])

        # ✅ Reject if not educational
        if not is_educational(all_text):
            return "❌ PDF rejected. Not educational (e.g., lacks syllabus/chapter/question)", None

        text_chunks = chunk_text(all_text)
        embeddings = text_embedder.encode(text_chunks, batch_size=16)
        clear_conversation_memory()

        return f"✅ Processed {len(uploaded_files)} PDF(s) - {len(text_chunks)} chunks", {
            "text_chunks": text_chunks,
            "embeddings": embeddings,
            "full_text": all_text
        }
    except Exception as e:
        return f"❌ PDF error: {str(e)}", None

# ✅ LLM Q&A
async def query_llm(user_question, doc):
    global conversation_memory
    if not user_question.strip():
        clear_conversation_memory()
        return "🧹 Memory cleared", doc

    if not doc or not doc.get("text_chunks"):
        return "❌ Upload PDF first", doc

    conversation_memory.append({"role": "user", "content": user_question})
    if len(conversation_memory) > MAX_MEMORY_ITEMS:
        conversation_memory = conversation_memory[-MAX_MEMORY_ITEMS:]

    q_embed = text_embedder.encode([user_question])[0]
    sim_scores = cosine_similarity([q_embed], doc["embeddings"])[0]
    top_indices = sim_scores.argsort()[-3:][::-1]
    context = "\n".join([doc["text_chunks"][i] for i in top_indices])

    messages = [
        {"role": "system", "content": "You are a tutor. Answer only from the given document."},
        {"role": "user", "content": f"{context}\n\nQ: {user_question}"}
    ] + conversation_memory[:-1]

    try:
        async with httpx.AsyncClient(timeout=30.0) as client:
            res = await client.post(
                GROQ_API_URL,
                headers={"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"},
                json={"model": MODEL_NAME, "messages": messages, "temperature": 0.7, "max_tokens": 1000}
            )
            res.raise_for_status()
            reply = res.json()['choices'][0]['message']['content']
            conversation_memory.append({"role": "assistant", "content": reply})
            return reply, doc
    except Exception as e:
        return f"❌ API error: {str(e)}", doc

# ✅ QUIZ GENERATION
async def generate_quiz(doc, topic, difficulty, mcq, short, long):
    if not doc or not doc.get("full_text"):
        return "❌ Upload PDF first", doc

    prompt = f"""Based on the content below, create a quiz on \"{topic}\".
Level: {difficulty}
Generate:
- {mcq} MCQs
- {short} Short Answer
- {long} Long Answer
Format strictly:
=== MCQs ===\n1. Q?\na)\nb)\nc)\nd)
=== SHORT ===\n1. Q?
=== LONG ===\n1. Q?
Content:\n{doc['full_text'][:4000]}
"""
    messages = [
        {"role": "system", "content": "You are a quiz generator."},
        {"role": "user", "content": prompt}
    ]

    try:
        async with httpx.AsyncClient(timeout=60.0) as client:
            res = await client.post(
                GROQ_API_URL,
                headers={"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"},
                json={"model": MODEL_NAME, "messages": messages, "temperature": 0.5, "max_tokens": 2000}
            )
            res.raise_for_status()
            return res.json()['choices'][0]['message']['content'], doc
    except Exception as e:
        return f"❌ Quiz error: {str(e)}", doc



# ✅ PDF GENERATION FUNCTIONS
def create_pdf_from_text(content, filename_prefix="document"):
    """Create a PDF from text content and return the file path"""
    try:
        # Create a temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf", prefix=filename_prefix)
        temp_path = temp_file.name
        temp_file.close()

        # Create PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        # Split content into lines and add to PDF
        lines = content.split('\n')
        for line in lines:
            # Handle long lines by wrapping them
            if len(line) > 80:
                words = line.split(' ')
                current_line = ""
                for word in words:
                    if len(current_line + word) < 80:
                        current_line += word + " "
                    else:
                        if current_line:
                            pdf.cell(0, 10, txt=current_line.encode('latin-1', 'ignore').decode('latin-1'), ln=True)
                        current_line = word + " "
                if current_line:
                    pdf.cell(0, 10, txt=current_line.encode('latin-1', 'ignore').decode('latin-1'), ln=True)
            else:
                pdf.cell(0, 10, txt=line.encode('latin-1', 'ignore').decode('latin-1'), ln=True)

        # Save the PDF
        pdf.output(temp_path)
        return temp_path
    except Exception as e:
        print(f"PDF creation error: {e}")
        return None

def save_quiz_as_pdf(quiz_content):
    """Save quiz content as PDF"""
    if not quiz_content or quiz_content.startswith("❌"):
        return None

    return create_pdf_from_text(quiz_content, "quiz_")

# ✅ GRADIO UI
with gr.Blocks(title="EduPDF Tutor & Quiz Generator") as app:
    doc_state = gr.State()

    with gr.Tab("📄 Q&A"):
        pdfs = gr.File(file_types=['.pdf'], file_count="multiple", type="binary", label="Upload PDFs")
        upload_status = gr.Textbox(label="Status")
        question = gr.Textbox(label="Ask a Question")
        ask_btn = gr.Button("Ask")
        answer_box = gr.Textbox(label="Answer", lines=10)

    with gr.Tab("📝 Generate Quiz"):
        topic = gr.Textbox(label="Topic")
        level = gr.Dropdown(choices=["easy", "medium", "hard"], value="medium", label="Difficulty")
        mcq = gr.Number(label="MCQs", value=3)
        short = gr.Number(label="Short", value=2)
        long = gr.Number(label="Long", value=1)

        gen_btn = gr.Button("Generate Quiz")
        quiz_box = gr.Textbox(label="Quiz Output", lines=15)

        save_btn = gr.Button("Save Quiz as PDF")
        pdf_output = gr.File(label="Quiz PDF")

    # Event handlers for Q&A tab
    pdfs.change(process_uploaded_pdfs, inputs=pdfs, outputs=[upload_status, doc_state])
    ask_btn.click(lambda q, d: asyncio.run(query_llm(q, d)), [question, doc_state], [answer_box, doc_state])

    # Event handlers for Quiz tab
    gen_btn.click(lambda d, t, l, m, s, lo: asyncio.run(generate_quiz(d, t, l, m, s, lo)), [doc_state, topic, level, mcq, short, long], [quiz_box, doc_state])
    save_btn.click(save_quiz_as_pdf, inputs=[quiz_box], outputs=[pdf_output])

app.launch()